# Workspace for Project Euler problems

## Problem 7

By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.

What is the 10 001st prime number?

In [1]:
import primetools as pt

In [2]:
pt.findSomePrimes(20)

{2: 0, 3: 0, 5: 0, 7: 0, 11: 0, 13: 0, 17: 0, 19: 0}

In [3]:
pt.findSomePrimes(number=20)

{2: 0,
 3: 0,
 5: 0,
 7: 0,
 11: 0,
 13: 0,
 17: 0,
 19: 0,
 23: 0,
 29: 0,
 31: 0,
 37: 0,
 41: 0,
 43: 0,
 47: 0,
 53: 0,
 59: 0,
 61: 0,
 67: 0,
 71: 0}

In [5]:
help(pt.findSomePrimes)

Help on function findSomePrimes in module primetools:

findSomePrimes(limit=0, number=0)
    Method that returns a map where each key is a prime number. The value of the map will be initialized to 0. This is for counting prime factors of some other number.



In [7]:
sp=pt.findSomePrimes(number=10001)

In [23]:
print(type(sp))
print(type(sp.keys()))
print(dir(sp.keys()))

spkeys=list(sp.keys())
spkeys.reverse()
print(type(spkeys))
print (dir(spkeys))

print (spkeys[0])

<class 'dict'>
<class 'dict_keys'>
['__and__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__rand__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__rsub__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__xor__', 'isdisjoint']
<class 'list'>
['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'exte